# Training A Model

In [7]:
!refit_init

Refit Installed, You can now run the following to import REFIT
from refit import Refit


In [8]:
#Example Notebook
from sklearn.datasets import load_iris

import datetime
from refit import Refit
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error
from refit.enums.model_format import ModelFormat
from skl2onnx.common.data_types import FloatTensorType, BooleanTensorType
from feature_extractor import FeatureExtractor

import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import matplotlib.pyplot as plt
import numpy as np

In [9]:
project_guid = "e2b6e5b3-cab6-47cc-a753-e655d3ac8ef1"
refit = Refit(project_guid)

start = datetime.datetime(2023, 6, 15, 0)
end = datetime.datetime(2023, 6, 16, 0)
sensors = ['EURtoUSD']

In [10]:
pd.set_option("display.max_columns", None)

In [ ]:
# Getting Sensor Data with the training window flag incorporated
df = refit.sensor_data(
    start=start, 
    end=end, 
    sensors=sensors, 
    feature_extractor=None, 
    include_flag=False
)
df

In [ ]:
feature_columns = ['usd_val']
features = df 
x = features[feature_columns]
y = 2*(features.usd_val.astype(float))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)


In [ ]:
y_pred

In [30]:
initial_types = [('input', FloatTensorType([None,len(feature_columns)]))]

In [31]:
refit.save(clf, feature_columns, ModelFormat.SK_LEARN, initial_types=initial_types)

'Model Published'

## test locally 

In [ ]:
import onnxruntime as rt
import numpy
# Load the ONNX model
sess = rt.InferenceSession("models/a816c3c8-bc2f-47fd-9841-8a108a5c78d2.onnx")
X_test = numpy.array(X_test)
input_name = sess.get_inputs()[0].name
X_test = X_test.astype(numpy.float32)
sess.run(None, {input_name: X_test})[0]